In [21]:
import os
import pandas as pd
import requests
import json
from itertools import combinations, product
from joblib import Parallel, delayed
import time
from typing import List, Dict, Any, Tuple
import random
from tqdm.auto import tqdm

In [22]:
OPENROUTER_API_KEY = "your-api-key-here"  # Replace with your actual API key
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
MODEL_NAME = "qwen/qwen3-235b-a22b-thinking-2507"

In [23]:
MAX_NUM_PROBLEMS = 2
BLOOM_LEVELS = ["จำ", "เข้าใจ", "นำไปใช้", "วิเคราะห์", "ประเมิน", "สร้างสรรค์"]
LEVELS = ["ง่าย", "ปานกลาง", "ยาก"]
GRADE_LEVELS = ["ม.4", "ม.5", "ม.6"]
TOPICS = ["พีชคณิต"]
QUESTION_TYPES = [
    "multiple_choice",
    # "short_answer"
    ]
ADDITIONAL_REQUIREMENTS = "โจทย์จำเป็นต้องมีคำตอบ และถ้าโจทย์เป็นแบบ multiple choice (ปรนัย) ต้องมีคำตอบหลอกจำนวน 3 ข้อ (ทั้งหมด หลอก + จริง มี 4 ข้อ) โดยมาจากการคำนวนที่ผิดพลาด"

# For parallel processing
N_JOBS = 4  # Adjust based on your system and API rate limits
BATCH_SIZE = N_JOBS * 2
DELAY_BETWEEN_REQUESTS = 1  # seconds

# ------------------------------------------------------------------------------

# Generate Bloom level combinations (1-2 levels max by default, can be increased)
BLOOM_COMBINATIONS = []
for r in range(1, 4):  # 1 to 3 bloom levels per question
    BLOOM_COMBINATIONS.extend(combinations(BLOOM_LEVELS, r))

# Convert combinations to list of lists for easier handling
BLOOM_COMBINATIONS = [list(combo) for combo in BLOOM_COMBINATIONS]

NUM_PROBLEMS = list(range(1, MAX_NUM_PROBLEMS + 1))

# Create all possible combinations of parameters
all_combinations = list(product(
    TOPICS,                   # topic
    GRADE_LEVELS,             # grade level
    QUESTION_TYPES,           # question type
    LEVELS,                   # difficulty
    BLOOM_COMBINATIONS,       # bloom levels
    NUM_PROBLEMS,             # num problems
))

print(f"Total possible combinations: {len(all_combinations)}")
print(f"Bloom combinations: {len(BLOOM_COMBINATIONS)}")
print(f"Sample combinations:")
for i, combo in enumerate(all_combinations[-5:]):
    topic, grade, q_type, difficulty, bloom, num_problems = combo
    print(f"  {i+1}. Topic: {topic}, Grade: {grade}, Type: {q_type}, Difficulty: {difficulty}, Bloom: {bloom}, Num Problems: {num_problems}")

Total possible combinations: 738
Bloom combinations: 41
Sample combinations:
  1. Topic: พีชคณิต, Grade: ม.6, Type: multiple_choice, Difficulty: ยาก, Bloom: ['นำไปใช้', 'วิเคราะห์', 'สร้างสรรค์'], Num Problems: 2
  2. Topic: พีชคณิต, Grade: ม.6, Type: multiple_choice, Difficulty: ยาก, Bloom: ['นำไปใช้', 'ประเมิน', 'สร้างสรรค์'], Num Problems: 1
  3. Topic: พีชคณิต, Grade: ม.6, Type: multiple_choice, Difficulty: ยาก, Bloom: ['นำไปใช้', 'ประเมิน', 'สร้างสรรค์'], Num Problems: 2
  4. Topic: พีชคณิต, Grade: ม.6, Type: multiple_choice, Difficulty: ยาก, Bloom: ['วิเคราะห์', 'ประเมิน', 'สร้างสรรค์'], Num Problems: 1
  5. Topic: พีชคณิต, Grade: ม.6, Type: multiple_choice, Difficulty: ยาก, Bloom: ['วิเคราะห์', 'ประเมิน', 'สร้างสรรค์'], Num Problems: 2


In [24]:
SYSTEM_PROMPT = """
คุณคือ ดร.อเล็กซานเดอร์ โอเลอร์ นักคณิตศาสตร์อันดับหนึ่งของโลกที่มีชื่อเสียงระดับโลกในด้านการวิจัยคณิตศาสตร์ขั้นสูง คุณได้รับรางวัลฟิลด์สเมดัลและมีผลงานตีพิมพ์ในวารสารคณิตศาสตร์ชั้นนำกว่า 200 บทความ คุณเชี่ยวชาญในทุกสาขาของคณิตศาสตร์ตั้งแต่พีชคณิต เรขาคณิต แคลคูลัส ไปจนถึงทฤษฎีจำนวนและคณิตศาสตร์ประยุกต์

ปัจจุบัน ดร.อเล็กซานเดอร์ กำลังทำหน้าที่เป็นครูสอนวิชาคณิตศาสตร์ที่โรงเรียนมัธยมปลายชื่อดังแห่งหนึ่ง โดยรับผิดชอบการสอนนักเรียนระดับชั้นมัธยมศึกษาปีที่ 4, 5 และ 6 (เทียบเท่า Grade 10, 11, 12) คุณมีปรัชญาในการสอนที่เชื่อว่าการเรียนรู้คณิตศาสตร์ต้องเป็นไปตามลำดับขั้นของการคิดที่เรียกว่า Bloom's Taxonomy เพื่อให้นักเรียนพัฒนาทักษะการคิดอย่างเป็นระบบ

หน้าที่หลักของคุณคือการสร้างโจทย์คณิตศาสตร์ที่มีคุณภาพสูง เหมาะสมกับระดับความสามารถของนักเรียน และสอดคล้องกับหลักการของ Bloom's Taxonomy อย่างเข้มงวด

## Bloom's Taxonomy ในการสอนคณิตศาสตร์

### 1. จำ (Remember)
**ความหมาย**: การจำและการเรียกคืนข้อมูล สูตร หรือขั้นตอนพื้นฐานที่ได้เรียนมาแล้ว
**คำสำคัญ**: จำได้, ระบุ, รายการ, ตั้งชื่อ, เลือก

### 2. เข้าใจ (Understand)
**ความหมาย**: การเข้าใจความหมาย แนวคิด และสามารถอธิบายด้วยคำพูดของตนเองได้
**คำสำคัญ**: อธิบาย, แปล, สรุป, เปรียบเทียบ, แสดงให้เห็น

### 3. นำไปใช้ (Apply)
**ความหมาย**: การนำความรู้ สูตร หรือขั้นตอนที่เรียนมาไปใช้แก้ปัญหาในสถานการณ์ใหม่
**คำสำคัญ**: คำนวณ, แก้, ใช้, แสดง, ดำเนินการ

### 4. วิเคราะห์ (Analyze)
**ความหมาย**: การแยกแยะองค์ประกอบ วิเคราะห์ความสัมพันธ์ และเข้าใจโครงสร้างของปัญหา
**คำสำคัญ**: แยกแยะ, เปรียบเทียบ, ตรวจสอบ, ทดสอบ, วิเคราะห์

### 5. ประเมิน (Evaluate)
**ความหมาย**: การตัดสิน ประเมินค่า ให้เหตุผล และแสดงความคิดเห็นโดยใช้เกณฑ์ที่กำหนด
**คำสำคัญ**: ตัดสิน, ประเมิน, วิจารณ์, แสดงความคิดเห็น, ให้เหตุผล

### 6. สร้างสรรค์ (Create)
**ความหมาย**: การสร้างสรรค์สิ่งใหม่ ออกแบบ วางแผน หรือสร้างโจทย์ปัญหาขึ้นมาเอง
**คำสำคัญ**: สร้าง, ออกแบบ, วางแผน, เสนอ, พัฒนา

## ตัวอย่างโจทย์ตาม Bloom's Taxonomy

### จำ (Remember)
โจทย์: สูตรหาพื้นที่วงกลมคือข้อใด
A) $ A = \pi r^2 $  B) $ A = 2\pi r $  C) $ A = \pi d $  D) $ A = \frac{1}{2}\pi r^2 $

### เข้าใจ (Understand)
โจทย์: อธิบายความหมายของ $ \frac{dy}{dx} = 3x^2 $ ในทางเรขาคณิต

### นำไปใช้ (Apply)
โจทย์: จงหาค่า $ x $ จากสมการ $ 2x + 5 = 13 $

### วิเคราะห์ (Analyze)
โจทย์: เปรียบเทียบพฤติกรรมของฟังก์ชัน $ f(x) = x^2 $ และ $ g(x) = x^3 $ เมื่อ $ x > 1 $

### ประเมิน (Evaluate)
โจทย์: นักเรียนคนหนึ่งอ้างว่า $ \sqrt{a + b} = \sqrt{a} + \sqrt{b} $ เสมอ จงประเมินว่าข้อความนี้ถูกหรือผิด พร้อมให้เหตุผล

### สร้างสรรค์ (Create)
โจทย์: ออกแบบฟังก์ชันกำลังสองที่มีจุดยอดอยู่ที่ $ (2, -3) $ และผ่านจุด $ (0, 1) $

## ตัวอย่างโจทย์แบบผสม 5 โจทย์

### โจทย์ที่ 1 (เข้าใจ + นำไปใช้)
หากฟังก์ชัน $ f(x) = 2x - 3 $ จงหาค่า $ f(5) $ และอธิบายความหมายของผลลัพธ์

### โจทย์ที่ 2 (วิเคราะห์ + ประเมิน)
เปรียบเทียบวิธีแก้สมการ $ x^2 - 5x + 6 = 0 $ ด้วยการแยกตัวประกอบและสูตรกำลังสอง แล้วประเมินว่าวิธีใดมีประสิทธิภาพมากกว่า

### โจทย์ที่ 3 (นำไปใช้ + วิเคราะห์)
ร้านค้าแห่งหนึ่งขายสินค้าในราคา $ 100x - x^2 $ บาท เมื่อขาย $ x $ ชิ้น จงหาจำนวนสินค้าที่ขายได้เงินสูงสุด

### โจทย์ที่ 4 (เข้าใจ + สร้างสรรค์)
จากกราฟ $ y = \sin x $ จงสร้างฟังก์ชันใหม่ที่มีแอมพลิจูดเป็น 3 เท่า และอธิบายการเปลี่ยนแปลง

### โจทย์ที่ 5 (วิเคราะห์ + ประเมิน + สร้างสรรค์)
นักเรียนสองคนแก้โจทย์หาค่าสูงสุดของ $ f(x) = -x^2 + 4x + 1 $ ได้คำตอบต่างกัน คนหนึ่งได้ $ x = 2 $ อีกคนได้ $ x = -2 $ จงวิเคราะห์ว่าใครถูก ประเมินข้อผิดพลาด และสร้างวิธีตรวจสอบคำตอบ

---

## คำสั่งการทำงาน

เมื่อได้รับโจทย์ให้สร้างข้อสอบคณิตศาสตร์ ให้ทำตามขั้นตอนดังนี้:

### ขั้นตอนที่ 1: การคิดและวางแผน
ในแท็ก `<think></think>` ให้ทำการ:
1. **คิดโจทย์เป็นภาษาอังกฤษก่อน** - เพื่อให้ได้แนวคิดที่ชัดเจน
2. **ทดลองหาคำตอบ** - ตรวจสอบว่าโจทย์สามารถแก้ได้จริง
3. **หากแก้ไม่ได้ ให้เปลี่ยนโจทย์** - แล้วทดลองใหม่จนได้โจทย์ที่ดี
4. **เมื่อได้โจทย์ที่สมบูรณ์แล้ว** - สร้างตัวเลือกคำตอบสำหรับ multiple choice (หากต้องการ)
5. **ตรวจสอบความสอดคล้องกับ Bloom's taxonomy** ที่กำหนด

### ขั้นตอนที่ 2: การเขียนผลลัพธ์
หลังจาก `<think></think>` แล้ว ให้เขียนโจทย์ในรูปแบบ XML ตามตัวอย่างนี้:

```xml
<questions>
  <question>
    <text>โจทย์คณิตศาสตร์ที่มี KaTeX formatting เช่น $ 4x + 3 = 2x + 9 $</text>
    <type>multiple_choice</type>
    <options>
      <option>$ ตัวเลือก1 $</option>
      <option>$ ตัวเลือก2 $</option>
      <option>$ ตัวเลือก3 $</option>
      <option>$ ตัวเลือก4 $</option>
    </options>
    <correct_answer>$ คำตอบที่ถูก $</correct_answer>
    <explanation>
      ขั้นตอนที่ 1: อธิบายด้วย KaTeX เช่น $ 4x - 2x + 3 = 9 $
      ขั้นตอนที่ 2: $ 2x + 3 = 9 $
      ขั้นตอนที่ 3: $ 2x = 6 $
      ขั้นตอนที่ 4: $ x = 3 $
    </explanation>
    <score>2</score>
    <difficulty>ง่าย</difficulty>
    <bloom_levels>
      <level>เข้าใจ</level>
      <level>วิเคราะห์</level>
    </bloom_levels>
  </question>

  <question>
    <text>จงหาค่า $ x $ ที่ทำให้ $ 3x - 7 = 2x + 5 $</text>
    <type>short_answer</type>
    <correct_answer>$ x = 12 $</correct_answer>
    <explanation>
      ขั้นตอนที่ 1: นำ $ 2x $ ไปยังข้างซ้าย และ $ 7 $ ไปยังข้างขวา
      $ 3x - 2x = 5 + 7 $
      ขั้นตอนที่ 2: คำนวณ
      $ x = 12 $
    </explanation>
    <score>2</score>
    <difficulty>ปานกลาง</difficulty>
    <bloom_levels>
      <level>เข้าใจ</level>
    </bloom_levels>
  </question>
</questions>
```

## หมายเหตุสำคัญ

### รูปแบบการเขียน:
- **ใช้ KaTeX format** `$ ... $` สำหรับตัวเลข สูตร และนิพจน์คณิตศาสตร์ทั้งหมด
- **ใช้คำไทยสำหรับ Bloom's level**: จำ, เข้าใจ, นำไปใช้, วิเคราะห์, ประเมิน, สร้างสรรค์
- **สามารถมีหลายระดับ Bloom's**: `<bloom_levels><level>เข้าใจ</level><level>นำไปใช้</level></bloom_levels>`

### ระดับความยาก:
- **ง่าย**: โจทย์พื้นฐาน 1-2 ขั้นตอน (คะแนน 1-2)
- **ปานกลาง**: โจทย์ที่ต้องใช้ความคิด 3-4 ขั้นตอน (คะแนน 3-4)
- **ยาก**: โจทย์ซับซ้อน ต้องวิเคราะห์เชิงลึก 5+ ขั้นตอน (คะแนน 5+)

### การให้คะแนน:
- Score ควรสะท้อนระดับความยากและเวลาที่ใช้ในการแก้ปัญหา
- โจทย์ที่ต้องใช้ Bloom's level สูงควรได้คะแนนมากกว่า

คุณพร้อมที่จะสร้างโจทย์คณิตศาสตร์คุณภาพสูงตามที่ได้รับมอบหมายแล้วหรือไม่?"""

In [25]:
def create_user_prompt(topic: str, grade_level: str, question_type: str, difficulty: str, bloom_levels: List[str], num_problems: int = 1, additional_requirements: str = ADDITIONAL_REQUIREMENTS) -> str:
    """
    Create user prompt for generating math problems

    Args:
        topic: The math topic (e.g., "พีชคณิต")
        grade_level: Grade level (e.g., "ม.4")
        question_type: Type of question (e.g., "multiple_choice", "short_answer")
        difficulty: Difficulty level (e.g., "ง่าย", "ปานกลาง", "ยาก")
        bloom_levels: List of Bloom taxonomy levels
        num_problems: Number of problems to generate
        additional_requirements: Additional requirements for the problem

    Returns:
        Formatted user prompt string
    """
    bloom_str = ", ".join(bloom_levels)

    prompt = f"""จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้
1. หัวข้อ: {topic}
2. สำหรับนักเรียน: {grade_level}
3. รูปแบบ: {question_type}
4. ความยาก: {difficulty}
5. bloom level: {bloom_str}
6. จำนวน: {num_problems} ข้อ
7. เพิ่มเติม: {additional_requirements}"""

    return prompt

In [26]:
def test_prompt_generation():
    """Test the prompt generation function"""
    sample_prompt = create_user_prompt(
        topic="พีชคณิต",
        grade_level="ม.4",
        question_type="multiple_choice",
        difficulty="ยาก",
        bloom_levels=["เข้าใจ", "นำไปใช้"],
        num_problems=1
    )
    print("Sample generated prompt:")
    print("-" * 50)
    print(sample_prompt)
    print("-" * 50)

test_prompt_generation()

Sample generated prompt:
--------------------------------------------------
จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้
1. หัวข้อ: พีชคณิต
2. สำหรับนักเรียน: ม.4
3. รูปแบบ: multiple_choice
4. ความยาก: ยาก
5. bloom level: เข้าใจ, นำไปใช้
6. จำนวน: 1 ข้อ
7. เพิ่มเติม: โจทย์จำเป็นต้องมีคำตอบ และถ้าโจทย์เป็นแบบ multiple choice (ปรนัย) ต้องมีคำตอบหลอกจำนวน 3 ข้อ (ทั้งหมด หลอก + จริง มี 4 ข้อ) โดยมาจากการคำนวนที่ผิดพลาด
--------------------------------------------------


In [27]:
def call_openrouter_api(user_prompt: str, system_prompt: str, model: str = MODEL_NAME):
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }

    data = json.dumps({
        "model": model,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        # "provider": {
        #     "order": ["deepinfra/fp8", "cerebras"]
        # },
        "temperature": 0.6,
        "top_p": 0.95,
        "top_k": 20,
        "max_tokens": 10000
    })

    try:
        response = requests.post(f"{OPENROUTER_BASE_URL}/chat/completions", headers=headers, data=data)

        if response.status_code == 200:
            response_json = response.json()
            return {
                "success": True,
                "error": "",
                "content": response_json["choices"][0]["message"]["content"],
                "reasoning": response_json["choices"][0]["message"]["reasoning"]
            }

        return {
            "success": False,
            "error": f"Request failed with status code: {response.status_code}",
            "content": "",
            "reasoning": ""
        }

    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "content": "",
            "reasoning": ""
        }

In [28]:
def generate_problem(params: Tuple[str, str, str, str, List[str], int]) -> Dict[str, Any]:
    topic, grade, q_type, difficulty, bloom, num_problems = params

    time.sleep(random.uniform(0.5, 1.5))

    user_prompt = create_user_prompt(
        topic=topic,
        grade_level=grade,
        question_type=q_type,
        difficulty=difficulty,
        bloom_levels=bloom,
        num_problems=num_problems
    )

    api_result = call_openrouter_api(user_prompt, SYSTEM_PROMPT)

    result = {
        "topic": topic,
        "grade": grade,
        "question_type": q_type,
        "difficulty": difficulty,
        "bloom": bloom,
        "num_problems": num_problems,
        "user_prompt": user_prompt,
        "success": api_result.get("success", False),
        "error": api_result.get("error", ""),
        "content": api_result.get("content", ""),
        "reasoning": api_result.get("reasoning", "")
    }

    return result

In [29]:
generate_problem(all_combinations[0])

{'topic': 'พีชคณิต',
 'grade': 'ม.4',
 'question_type': 'multiple_choice',
 'difficulty': 'ง่าย',
 'bloom': ['จำ'],
 'num_problems': 1,
 'user_prompt': 'จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้\n1. หัวข้อ: พีชคณิต\n2. สำหรับนักเรียน: ม.4\n3. รูปแบบ: multiple_choice\n4. ความยาก: ง่าย\n5. bloom level: จำ\n6. จำนวน: 1 ข้อ\n7. เพิ่มเติม: โจทย์จำเป็นต้องมีคำตอบ และถ้าโจทย์เป็นแบบ multiple choice (ปรนัย) ต้องมีคำตอบหลอกจำนวน 3 ข้อ (ทั้งหมด หลอก + จริง มี 4 ข้อ) โดยมาจากการคำนวนที่ผิดพลาด',
 'success': True,
 'error': '',
 'content': '```xml\n<questions>\n  <question>\n    <text>สูตรการหาผลบวกของลำดับเลขคณิต $ n $ พจน์แรก คือข้อใด?</text>\n    <type>multiple_choice</type>\n    <options>\n      <option>$ S_n = \\frac{n}{2} (2a + (n-1)d) $</option>\n      <option>$ S_n = \\frac{n}{2} (a + nd) $</option>\n      <option>$ S_n = \\frac{n}{2} (2a + nd) $</option>\n      <option>$ S_n = n \\times (a + d) $</option>\n    </options>\n    <correct_answer>$ S_n = \\frac{n}{2} (2a + (n-1)d) $</correct_

In [20]:
all_results = []

In [ ]:
print(f"Generating {len(combinations)} problems using {N_JOBS} parallel jobs...")

progress = tqdm(range(0, len(combinations[:20]), BATCH_SIZE), desc="Progress")
for i in progress:
    batch = combinations[:20][i:i+BATCH_SIZE]

    batch_results = Parallel(n_jobs=N_JOBS, verbose=0)(
        delayed(generate_problem)(problem) for problem in batch
    )

    all_results.extend(batch_results)

    time.sleep(DELAY_BETWEEN_REQUESTS)

print("Done!")

In [ ]:
df = pd.DataFrame(all_results)

timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")

df.to_csv(f"problems_{timestamp}.csv", index=False, encoding='utf-8-sig')
df.to_parquet(f"problems_{timestamp}.parquet", index=False)